# 以下内容来自<<深度学习之PyTorch实战计算机视觉>>
唐进民 著

In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import torch
#torch.FloatTensor用于生成数据类型为浮点型的Tensor，传递给torch.FloatTensor的参数可以是一个列表也可以是一个维度值。
a = torch.FloatTensor([1.1,2.1,3.1,4.1,5.1])
b = torch.IntTensor([1,2,3,4,5])
c = torch.rand(2,3)
d = torch.randn(2,3) #符合正态分布
#e = torch.range(start,end,step)
#torch.zeros()

In [ ]:
#torch的运算
a_abs = torch.abs(a)
c_plus_d   = torch.add(c,d)
#如果小于下边界重写成下边界，上边界亦是如此
b_clamp = torch.clamp(b,-1,3)
c_div_d = torch.div(c,d) #除法
c_mul_d = torch.mul(c,d) #逐项乘法
a_squared = torch.pow(a,2)
f = torch.rand(3,2)
d_mm_f = torch.mm(d,f) #矩阵乘法

In [ ]:
#先通过输入的Tensor数据类型的变量在神经网络的前向传播过程中生成一张计算图，然后根据这个计算图和输出结果准确计算出每个参数需要更新的梯度，并反向传播完成梯度更新。
#在实践中需要用到Variable类对我们定义的Tensor数据类型进行封装，在封装后，计算图的每个节点就是一个Variable对象。
#如果用X表示结点，X.Tensor表示变量，X.grad表示梯度
#==============================================================================================
#                          采用自动梯度的方法自定义简单的二层网络
#==============================================================================================

from torch.autograd import Variable
batch_n = 100 #在一个批次中输入数据的数量
hidden_layer = 100 
input_data = 1000
output_data = 10

epoch_n = 20
learning_rate = 1e-6
for epoch in range(epoch_n):
    y_pred = x.mm(w1).clamp(min=0).mm(w2) #用了一层RELU这是？ 一层FCN，一层RELU，一层FCN，NB！
    loss = (y_pred - y).pow(2).sum()
    print('epoch:{},loss:{:.4f}'.format(epoch,loss.data[0]))
    
    loss.backward()
    
    w1.data = learning_rate * w1.grad.data
    w2.data = learning_rate * w2.grad.data
    
    w1.grad.data.zero_()
    w2.grad.data.zero_()

In [ ]:
#==============================================================================================
#                       通过构建一个继承了torch.nn.Module的新类实现前向传播和后向传播
#==============================================================================================

#报错提醒，对于高版本的pytorch，不能使用loss.data[0],而应该实用loss.item()取而代之


import torch
from torch.autograd import Variable
batch_n = 64 #在一个批次中输入数据的数量
hidden_layer = 100 
input_data = 1000
output_data = 10

class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
    def forward(self,input,w1,w2):
        x = torch.mm(input,w1)
        x = torch.clamp(x,min=0)
        x = torch.mm(x,w2)
        return x
    def backward(self):
        pass
    #后向传播如果没有特别的需求，则在一般情况下不进行调整。
model = Model()

#准备好数据集开始训练啦！
x = Variable(torch.randn(batch_n,input_data),requires_grad=False)
y = Variable(torch.randn(batch_n,output_data),requires_grad=False)

w1 = Variable(torch.randn(input_data,hidden_layer),requires_grad = True)
w2 = Variable(torch.randn(hidden_layer,output_data),requires_grad = True)

epoch_n = 30
learning_rate = 1e-6

for epoch in range(epoch_n):
    y_pred = model(x,w1,w2)
    loss = (y_pred - y).pow(2).sum()
    print('epoch:{},loss:{:.4f}'.format(epoch,loss.item()))
    loss.backward()
    
    w1.data -= learning_rate * w1.grad.data
    w2.data -= learning_rate * w2.grad.data
    
    w1.grad.data.zero_()
    w2.grad.data.zero_()

In [ ]:
#torch.nn包里面有很多帮助快速搭建网络的工具：比如CNN的卷积层，池化层，全连接层，防止过拟合参数的归一化方法，Dropout方法
#激活函数部分的线性激活函数，非线性激活函数的相关方法.
import torch
from torch.autograd import Variable
batch_n = 64 #在一个批次中输入数据的数量
hidden_layer = 100 
input_data = 1000
output_data = 10
x = Variable(torch.randn(batch_n,input_data),requires_grad=False)
y = Variable(torch.randn(batch_n,output_data),requires_grad=False)

#torch.nn.Sequential是torch.nn的一种序列容器，通过在容器中嵌套各种实现神经网络中具体功能相关的类，完成网络的搭建
#最主要的是，参数会按照我们定义好的序列自动传递下去。！！！容器中模块的加入有两种方式，其一：直接嵌套。其二：以为orderdict有序字典的方式传入
#后者搭建的模型每个模块都有我们给它起的名字，而前者默认使用从零开始的数字序列作为每个模块的名字。

#TYPR1:直接嵌套
models_TYPE1 = torch.nn.Sequential(
   torch.nn.Linear(input_data,hidden_layer),
   torch.nn.ReLU(),
   torch.nn.Linear(hidden_layer,output_data)
)
print(models_TYPE1)

#TYPE2:使用orderdict有序字典进行传入
from collections import OrderedDict
models_TYPE2 = torch.nn.Sequential(
    OrderedDict([
        ('Line1',torch.nn.Linear(input_data,hidden_layer)),
        ('Relu1',torch.nn.ReLU()),
        ('Line2',torch.nn.Linear(hidden_layer,output_data))
                ])
                                  )
print(models_TYPE2)

#使用orderdict的方法可以让我们更便捷的找到模型中相应的模块并进行操作。
#torch.nn.Linear(inputsize,outputsize,use_bias?) 在定义这一层时，函数内部已经完成了对于这一层参数的初始化




In [ ]:
#激活函数介绍: torch.nn.ReLU
#            torch.nn.Tanh
#            torch.nn.LeakyReLU
#            torch.nn.Sigmoid
#            torch.nn.Softmax

import torch
from torch.autograd import Variable
loss_f = torch.nn.MSELoss()  #MSELoss常常用于回归问题
#损失函数介绍
#            torch.nn.L1Loss  两个输入维度一致
#            torch.nn.CrossEntropyLoss 常常用于分类问题，第一个参数的维度是(a,b) 另一个是(a)
#            CrossEntropyLoss函数是把结果先进性softmax归一化然后再放到传统的softmax里面
#            两个参数分别是：a个样本对于b种label的值、a个样本实际的label
#            torch.nn.LeakyReLU



#随机生成了两个Tensor，然后求取MSE
x = Variable(torch.randn(100,100))
y = Variable(torch.randn(100,100))
loss = loss_f(x,y)
#print(loss.data)

#随机生成了两个Tensor，然后求取CrossEntropyLoss.
#注意，第二个参数一定得是torch.LongTensor!!!!!!

loss_f = torch.nn.CrossEntropyLoss()
x = Variable(torch.randn(5,3)) #5个样本，2种类label。
y = Variable(torch.LongTensor([1,0,1,0,1]))
loss = loss_f(x,y)

epoch_n = 10000
input_data = 1000
output_data = 10
#print(loss.data)

x = Variable(torch.randn(batch_n,input_data),requires_grad=False)
y = Variable(torch.randn(batch_n,output_data),requires_grad=False)
loss_fn = torch.nn.MSELoss()
for epoch in range(epoch_n):
    y_pred = models_TYPE2(x)
    loss = loss_fn(y_pred,y)
    if epoch % 1000 == 0:
        print('epoch:{},loss:{}'.format(epoch,loss.item()))
    models_TYPE2.zero_grad()
    loss.backward()
    for param in models_TYPE2.parameters():
        param.data -= param.grad.data * learning_rate



In [ ]:
#使用高级的优化器，而不是简单的全局梯度下降
#在torch.optim包里提供了非常多的可实现参数自动化的类，比如:SGD,AdaGrad,RMSProp,Adam等
#=============================================================================
#                       完整的基于torch的神经网络
#=============================================================================

#神文：https://blog.csdn.net/sinat_16643223/article/details/86670137
#只有使用SGD才考虑batch,用于SGD的一批数据量就是1个batch_n, set_size = batch_n * batch_size
#                                                     batch_size = 1时，就是纯正的梯度下降，当你的GPU足够牛批时，batch_size越小越好.
#一个batch size地样本走一遍后，每个样本都会算出一个损失函数值，把这些损失函数值相加再处以batch size值可以得出这一个batch地平均损失函数值，最后是拿这个平均损失函数值去梯度下降更新参数值。
#epoch=n  代表把整个训练集训练n遍。

import torch
from torch.autograd import Variable

batch_n = 100
input_data = 1000
hidden_layer = 1000
output_data = 10

x = Variable(torch.randn(batch_n,input_data),requires_grad = False)
y = Variable(torch.randn(batch_n,output_data),requires_grad = False)

models = torch.nn.Sequential(
         torch.nn.Linear(input_data,hidden_layer),
         torch.nn.ReLU(),
         torch.nn.Linear(hidden_layer,output_data)
)

epoch_n = 10000
learning_rate = 1e-4
loss_f = torch.nn.MSELoss()
optimizer = torch.optim.Adam(models.parameters(),lr=learning_rate) #第一个参量是被优化的参数，第二个参量是学习率的初始值

#开始训练
for epoch in range(epoch_n):
    y_pred = models(x)
    loss = loss_f(y_pred,y)
    if epoch % 100 == 0:
        print('epoch:{},loss:{}'.format(epoch,loss.item()))
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
        

In [ ]:
#实战MNIST
#具体过程：模型训练 -> 模型测试
#torch.nn实现网络结构,激活函数，损失函数。torch.optim是优化器，torch.autograd实现自动有梯度下降

#torchvision包的主要功能是实现数据的导入，预处理，预览等。

import torch
import torchvision
from torchvision import datasets,transforms #datasets用于下载数据集，transforms提供了丰富的类对载入的数据进行变换，下载让我们看看如何变幻
from torch.autograd import Variable
import matplotlib.pyplot as plt
#常用的数据包COCO,ImageNet,CIFAR都可以通过这个函数下载

transform = transforms.Compose([transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])])

#tips:一定要注意transformer的维度问题！！！不然会报出[1,28,28] cannot broadcast to [3,28,28]
#对于MNIST可以不用数据增强，因为对模型训练来说已经足够了
#我们可以把transforms.Compose看作是和torch.nn.Sequential一样的容器,可以装很多数据预处理模块,传入的参量是一个列表。
#对于torchvison常用的数据变换操作的介绍我放在Picture_process模块中介绍。

data_train = datasets.MNIST(root='./',transform=transform,train=True,download=True)
data_test = datasets.MNIST(root='./',transform=transform,train=False,download=True)

batch_size = 64

data_loader_train = torch.utils.data.DataLoader(dataset=data_train,batch_size=batch_size,shuffle=True)
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,batch_size=batch_size,shuffle=True)
#data_loader_train是一个可迭代对象，直接拿next(iter())取走即可
#torchvision.utils.make_grid的参数就是一个批次的装载数据

images,labels = next(iter(data_loader_train)) #data_loader这个可迭代对象是一个四维的东西。
print(images.shape)
img = torchvision.utils.make_grid(images)
#torchvision.utils.make_grid 最多传入四个参数，batch_size,channel,height,weight

img = img.numpy().transpose(1,2,0)
std = [0.5,0.5,0.5]
mean = [0.5,0.5,0.5]
img = img*std + mean
print([labels[i] for i in range(64)])
plt.imshow(img)
#用plt打印需要注意必须必须必须传入(height,weight,channel)

In [ ]:
class Model(torch.nn.Module):
    def __init__(self):
        super(Model,self).__init__()
        self.conv1 = torch.nn.Sequential(
                     torch.nn.Conv2d(1,64,kernel_size=3,stride=1,padding=1),
                     torch.nn.ReLU(),
                     torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
                     torch.nn.ReLU(),
                     torch.nn.MaxPool2d(stride=2,kernel_size=2)  )#池化层第一个参数是步长，第二个参数位置是池化核大小
        
        self.dense = torch.nn.Sequential(
                     torch.nn.Linear(128*14*14,1024),
                     torch.nn.ReLU(),
                     #dropout每epoch时选择丢弃的神经连接是不同的，这样做是为了让我们最后训练出的模型对于各部分的参数不产生过度依赖
                     torch.nn.Dropout(p=0.5),#dropout默认0.5
                     torch.nn.Linear(1024,10)                               )
        
    def forward(self,x):
        x = self.conv1(x)
        x = x.view(-1,128*14*14) #扁平化，从conv1层输出的的是14*14的128通道的feature_map
        x = self.dense(x)
        return x
    
model = Model()
print('========================================================================================')
print(model)
print('========================================================================================')

loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(model.parameters())     

n_epochs = 15

if torch.cuda.is_available():
    model = model.cuda()
    loss_f = loss_f.cuda()

for epoch in range(n_epochs):
    running_loss = 0.0
    running_correct = 0
    for data in data_loader_train:
        x_train,y_train = data
        x_train,y_train = Variable(x_train),Variable(y_train)
        x_train = x_train.cuda()
        y_train = y_train.cuda()
        outputs = model(x_train)
        _,pred = torch.max(outputs.data,1)
        optimizer.zero_grad()
        loss = loss_f(outputs,y_train)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
        running_correct += torch.sum(pred==y_train.data)
    print('epoch is:{},loss is:{}'.format(epoch,loss))

    #测试啦!!!
    testing_correct = 0
    for data in data_loader_test:
        x_test,y_test = data
        x_test,y_test = Variable(x_test),Variable(y_test)
        x_test = x_test.cuda()
        y_test = y_test.cuda()
        outputs = model(x_test)
        _,pred = torch.max(outputs.data,1)
        testing_correct += torch.sum(pred == y_test.data)
    print("CORRECT IS:{}".format(testing_correct/len(data_test)))
    

In [ ]:
torch.save(model.state_dict(),'./')

In [ ]:
#summary(model,(3,32,32))非常牛批直接查看网络结构
#pip install torchsummary

#随便找几个图儿意思意思
import matplotlib.pyplot as plt 
data_loader_test = torch.utils.data.DataLoader(dataset=data_test,batch_size=4,shuffle=True)
x_test,y_test = next(iter(data_loader_test))
inputs = Variable(x_test)
inputs = inputs.cuda()
pred = model(inputs)
_,pred = torch.max(pred,1)

print('predict label is:',[i for i in pred.data])
print('real label is:',[i for i in y_test])

img = torchvision.utils.make_grid(x_test)
img = img.numpy().transpose(1,2,0)
std = [0.5,0.5,0.5]
mean = [0.5,0.5,0.5]
img = img*std + mean
plt.imshow(img)


model = Model()   
model = model.cuda()    
summary(model,(3,32,32))   
!!!直接查看各层featuremap大小

In [ ]:
#tensor转list
import torch
import numpy as np

a = torch.FloatTensor([1.0,2.0,3.0,4.0,5.0])
a = a.cuda()
a = np.array(a.cpu())
a = a.tolist()

# 迁移学习（猫狗大战）

In [ ]:
#对于原始数据比较少的问题，我们可以使用迁移学习进行有效的解决。
#我们通过对一个训练好的模型进行细微调整，就能将其应用到相似的问题中。
import torch
import torchvision
from torchvision import datasets,transforms
import os
import numpy as np
import matplotlib.pyplot as plt
import time


#数据预处理
#torchvision.ImageFolder是一个通用的数据集加载器，他要求我们以data/train/cat这种方式来存放图片，label会顺序返回的
#我们得到的dataset,它的结构就是[(img_data,class_id),(img_data,class_id),…] 
#注意torchvision.datasets.ImageFolder的使用格式鸭


data_dir = '../input/dogs-vs-cats/dataset'
data_transform = {x:transforms.Compose([transforms.Scale([64,64]),transforms.ToTensor()]) for x in ['training_set','test_set']}
image_datasets = {x:datasets.ImageFolder(root = os.path.join(data_dir,x),transform = data_transform[x]) for x in ['training_set','test_set']}
dataloader = {x:torch.utils.data.DataLoader(dataset = image_datasets[x],batch_size=16,shuffle=True) for x in ['training_set','test_set']}
#拼接文件名：   os.path.join
#目录下的文件名 os.path.dirname
#文件是否存在   os.path.exists
#是否是目录名   os.path.isdir
#是否是文件名   os.path.isfil
x_example,y_example = next(iter(dataloader['training_set']))

#展示标签及对应的label
#index_classes = image_datasets['training_set'].class_to_idx
#print(index_classes)

img = torchvision.utils.make_grid(x_example)
img = img.numpy().transpose([1,2,0])
#print([example_classes[i] for i in y_example])
plt.imshow(img)
plt.show()



**本节会先搭建一个简化的VGGNet，然后迁移一个完整的VGG16框架，最后迁移一个ResNet50架构**

In [ ]:
import torch
#这个网络是把paper中的VGGNet
#VGGNet删除了最后3个卷积层和1个池化层
class VGGNet(torch.nn.Module):
    def __init__(self):
        super(VGGNet,self).__init__()
        self.Conv = torch.nn.Sequential(
           torch.nn.Conv2d(3,64,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(64,64,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(kernel_size=2,stride=2),
            
           torch.nn.Conv2d(64,128,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(128,128,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(kernel_size=2,stride=2),
            
           torch.nn.Conv2d(128,256,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(256,256,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(kernel_size=2,stride=2),
            
           torch.nn.Conv2d(256,512,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.Conv2d(512,512,kernel_size=3,stride=1,padding=1),
           torch.nn.ReLU(),
           torch.nn.MaxPool2d(kernel_size=2,stride=2)    
        )
        
        self.Classes = torch.nn.Sequential(
           torch.nn.Linear(4*4*512,1024),
           torch.nn.ReLU(),
           torch.nn.Dropout(p=0.5),
           torch.nn.Linear(1024,2)
        )
    def forward(self,x):
        x = self.Conv(x)
        x = x.view(-1,4*4*512)
        x = self.Classes(x)
        return x

vggnet = VGGNet()
print('======================================================================================')
print(vggnet)
print('======================================================================================')

#定义损失函数和优化器
loss_f = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vggnet.parameters(),lr=1e-5)

epoch_n = 30

if torch.cuda.is_available(): 
    vggnet = vggnet.cuda() 
    loss_f = loss_f.cuda()
    
for epoch in range(epoch_n):
    print('Epoch {}/{}'.format(epoch,epoch_n-1))
    print('-'*20)
    for phase in ['training_set','test_set']:
        if phase == 'training_set':
            print('Training...')
            vggnet.train(True)
        else:
            print('Validing...')
            vggnet.train(False)
        running_loss = 0.0
        running_corrects= 0
    
        for batch,data in enumerate(dataloader[phase],1):
            x,y = data
            x,y = x.cuda(),y.cuda()
            y_pred = vggnet(x)
            _,pred = torch.max(y_pred.data,1)
            optimizer.zero_grad()
            loss = loss_f(y_pred,y)
        
            if phase == 'training_set':
               loss.backward()
               optimizer.step()
            running_loss += loss.item()
            running_corrects += torch.sum(pred == y.data)
        
            if batch%100 == 0 and phase =='training_set':
                print('Batch:{},Train Loss:{:.4f},Train ACC:{:.4f}'.format(batch,running_loss/batch,100*running_corrects/(16*batch)))
    
        epoch_loss = running_loss*16/len(image_datasets[phase])
        epoch_acc = 100 * running_corrects/len(image_datasets[phase])
        print('{} Loss:{:.4f} Acc:{:.4f}%'.format(phase,epoch_loss,epoch_acc))
    
      


# 迁移学习VGG16   (VGG大概比Alexnet慢5~6倍)

In [ ]:
#首先需要下载已经具备最优参数的模型，这需要对于之前的 vggnet = VGGNet() 进行替换，因为我们不再需要自己搭建和定义模型了，而是通过代码自动下载模型并直接调用。
from torchvision import models
vggnet_transfer = models.vgg16(pretrained=True)
#print('==================================================================')
#print(vggnet_transfer)#我们发现这个下载好的网络的输出参数是1000
#print('==================================================================')
#调整下载的模型，其基本思想是冻结卷积神经网络中全连接层之前的全部网络层次，让这些被冻结的网络层次中的参数在模型的训练过程中不进行参数的梯度更新
#能够被优化的参数仅仅是没有被冻结的全连接层的全部参数

#冻结操作
for param in vggnet_transfer.parameters():
    param.requires_grad = False

vggnet_transfer.classifier = torch.nn.Sequential(
                            torch.nn.Linear(25088,4096),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(4096,4096),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(4096,2)
)


cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(vggnet_transfer.classifier.parameters(),lr=1e-5)

if torch.cuda.is_available(): 
    vggnet_transfer = vggnet_transfer.cuda() 
    cost = cost.cuda()
    
print('==================================================================')
print(vggnet_transfer)#修改了FCN之后的vggnet_transfer
print('==================================================================')

epoch_n = 10

for epoch in range(epoch_n):
    print('Epoch {}/{}'.format(epoch,epoch_n-1))
    print('-'*20)
    for phase in ['training_set','test_set']:
        if phase == 'training_set':
            print('Training...')
            vggnet.train(True)
        else:
            print('Validing...')
            vggnet.train(False)
        running_loss = 0.0
        running_corrects= 0
    
        for batch,data in enumerate(dataloader[phase],1):
            x,y = data
            x,y = x.cuda(),y.cuda()
            y_pred = vggnet_transfer(x)
            _,pred = torch.max(y_pred.data,1)
            optimizer.zero_grad()
            loss = cost(y_pred,y)
        
            if phase == 'training_set':
               loss.backward()
               optimizer.step()
            running_loss += loss.item()
            running_corrects += torch.sum(pred == y.data)
        
            if batch%100 == 0 and phase =='training_set':
                print('Batch:{},Train Loss:{:.4f},Train ACC:{:.4f}'.format(batch,running_loss/batch,100*running_corrects/(16*batch)))
    
        epoch_loss = running_loss*16/len(image_datasets[phase])
        epoch_acc = 100 * running_corrects/len(image_datasets[phase])
        print('{} Loss:{:.4f} Acc:{:.4f}%'.format(phase,epoch_loss,epoch_acc))
    

# 迁移学习Resnet50

In [ ]:
from torchvision import models
resnet50_transfer = models.resnet50(pretrained=True)
#print(resnet50_transfer)

for param in resnet50_transfer.parameters():
    param.requires_grad = False


resnet50_transfer.fc = torch.nn.Sequential(
                            torch.nn.Linear(2048,2048),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(2048,2048),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(2048,2)
)
print('=============================================================================================')
print(resnet50_transfer)
print('=============================================================================================')

cost = torch.nn.CrossEntropyLoss()
optimizer = torch.optim.Adam(resnet50_transfer.fc.parameters(),lr=1e-5)

if torch.cuda.is_available(): 
    resnet50_transfer = resnet50_transfer.cuda() 
    cost = cost.cuda()

epoch_n = 10

for epoch in range(epoch_n):
    print('Epoch {}/{}'.format(epoch,epoch_n-1))
    print('-'*20)
    for phase in ['training_set','test_set']:
        if phase == 'training_set':
            print('Training...')
            vggnet.train(True)
        else:
            print('Validing...')
            vggnet.train(False)
        running_loss = 0.0
        running_corrects= 0
    
        for batch,data in enumerate(dataloader[phase],1):
            x,y = data
            x,y = x.cuda(),y.cuda()
            y_pred = resnet50_transfer(x)
            _,pred = torch.max(y_pred.data,1)
            optimizer.zero_grad()
            loss = cost(y_pred,y)
        
            if phase == 'training_set':
               loss.backward()
               optimizer.step()
            running_loss += loss.item()
            running_corrects += torch.sum(pred == y.data)
        
            if batch%100 == 0 and phase =='training_set':
                print('Batch:{},Train Loss:{:.4f},Train ACC:{:.4f}'.format(batch,running_loss/batch,100*running_corrects/(16*batch)))
    
        epoch_loss = running_loss*16/len(image_datasets[phase])
        epoch_acc = 100 * running_corrects/len(image_datasets[phase])
        print('{} Loss:{:.4f} Acc:{:.4f}%'.format(phase,epoch_loss,epoch_acc))

# 模型融合（比赛中常用）

In [ ]:
#我们在使用单一模型处理某个问题时，很容易遇到泛化瓶颈，模型的泛化能力因一些客观因素受到了限制。
#为了避免过于庞大耗时的融合模型，我们一般挑选一些结构表较简单，网络层次比较少的神经网络参与到多模型融合中，如果还想继续借用深层次网络来实现多模型融合就需要使用迁移学习方法来辅助模型的融合，以减少模型训练耗

# 模型融合分为过程融合和结果融合，本章仅讨论结果融合
# 结果融合有三种主要类型：多数表决，直接平均，加权平均
# 参与融合的各个模型在输出结果上表现得差异性越高，最终得到的融合模型的预测效果越好。
#对于原始数据比较少的问题，我们可以使用迁移学习进行有效的解决。
#我们通过对一个训练好的模型进行细微调整，就能将其应用到相似的问题中。
import torch
import torchvision
from torchvision import datasets,transforms
import os
import numpy as np
import matplotlib.pyplot as plt
import time


#数据预处理
#torchvision.ImageFolder是一个通用的数据集加载器，他要求我们以data/train/cat这种方式来存放图片，label会顺序返回的
#我们得到的dataset,它的结构就是[(img_data,class_id),(img_data,class_id),…] 
#注意torchvision.datasets.ImageFolder的使用格式鸭


data_dir = '../input/dogs-vs-cats/dataset'
data_transform = {x:transforms.Compose([transforms.Scale([64,64]),transforms.ToTensor()]) for x in ['training_set','test_set']}
image_datasets = {x:datasets.ImageFolder(root = os.path.join(data_dir,x),transform = data_transform[x]) for x in ['training_set','test_set']}
dataloader = {x:torch.utils.data.DataLoader(dataset = image_datasets[x],batch_size=16,shuffle=True) for x in ['training_set','test_set']}
#拼接文件名：   os.path.join
#目录下的文件名 os.path.dirname
#文件是否存在   os.path.exists
#是否是目录名   os.path.isdir
#是否是文件名   os.path.isfil
x_example,y_example = next(iter(dataloader['training_set']))

#展示标签及对应的label
#index_classes = image_datasets['training_set'].class_to_idx
#print(index_classes)

img = torchvision.utils.make_grid(x_example)
img = img.numpy().transpose([1,2,0])
#print([example_classes[i] for i in y_example])
plt.imshow(img)
plt.show()


In [ ]:
from torchvision import models
vgg16 = models.vgg16(pretrained=True)
resnet50 = models.resnet50(pretrained=True)

for param in vgg16.parameters():
    param.requires_grad = False
vgg16.classifier = torch.nn.Sequential(
                              torch.nn.Linear(25088,4096),
                              torch.nn.ReLU(),
                              torch.nn.Dropout(p=0.5),
                              torch.nn.Linear(4096,4096),
                              torch.nn.ReLU(),
                              torch.nn.Dropout(p=0.5),
                              torch.nn.Linear(4096,2)

)

for param in resnet50.parameters():
    param.requires_grad = False
resnet50.fc = torch.nn.Sequential(
                          torch.nn.Linear(2048,2048),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(2048,2048),
                            torch.nn.ReLU(),
                            torch.nn.Dropout(p=0.5),
                            torch.nn.Linear(2048,2)
)


vgg16 = vgg16.cuda()
resnet50 = resnet50.cuda()

loss_f_vgg16 = torch.nn.CrossEntropyLoss().cuda()
loss_f_resnet50 = torch.nn.CrossEntropyLoss().cuda()
loss_f_vgg16,loss_f_resnet50 = loss_f_vgg16.cuda(),loss_f_resnet50.cuda()
optimizer_vgg16 = torch.optim.Adam(vgg16.classifier.parameters(), lr=1e-5)
optimizer_resnet50 = torch.optim.Adam(resnet50.fc.parameters(), lr=1e-5)

weight_vgg16 = 0.6
weight_resnet50 = 0.4

epoch_n = 10

for epoch in range(epoch_n):
    for phase in ['training_set','test_set']:
        if phase == 'training_set':
            print('Training...')
            vgg16.train(True)
            resnet50.train(True)
        else:
            print('Validing...')
            vgg16.train(False)
            resnet50.train(False)
        running_loss_vgg16 = 0.0
        running_corrects_vgg16 = 0
        running_loss_resnet50 = 0.0
        running_corrects_resnet50 = 0
        blending_running_corrects = 0
        
        for batch,data in enumerate(dataloader[phase],1):
            x,y = data
            x,y = x.cuda(),y.cuda()
            y_pred_vgg16 = vgg16(x)
            y_pred_resnet50 = resnet50(x)
            blending_y_pred =  y_pred_vgg16*weight_vgg16 + y_pred_resnet50*weight_resnet50
        
            _,pred_vgg16 = torch.max(y_pred_vgg16.data,1)
            _,pred_resnet50 = torch.max(y_pred_resnet50.data,1)
            _,blending_pred = torch.max(blending_y_pred.data,1)
        
            optimizer_vgg16.zero_grad()
            optimizer_resnet50.zero_grad()
        
            loss_vgg16 = loss_f_vgg16( y_pred_vgg16,y)
            loss_resnet50 = loss_f_resnet50(y_pred_resnet50,y)
        
            if phase == 'training_set':
                loss_vgg16.backward()
                loss_resnet50.backward()
                optimizer_vgg16.step()
                optimizer_resnet50.step()
            
            running_loss_vgg16 += loss_vgg16.item()
            running_corrects_vgg16 += torch.sum(pred_vgg16 == y.data) 

            running_loss_resnet50 += loss_resnet50.item()
            running_corrects_resnet50 += torch.sum(pred_resnet50 == y.data)
        
            blending_running_corrects += torch.sum(blending_pred == y.data)
        
            if batch % 100 == 0 and phase == 'training_set':
                print('Batch:{}--vgg16_loss:{:.4f},vgg16_acc:{:.4f}--resnet50_loss:{:.4f},resnet50_acc:{:.4f}--blending_acc:{:.4f}'.format(batch,running_loss_vgg16/batch,100*running_corrects_vgg16/(16*batch),running_loss_resnet50/batch,100*running_corrects_resnet50/(16*batch),100*blending_running_corrects/(16*batch)))
        epoch_loss_vgg16 = running_loss_vgg16 * 16 / len(image_datasets[phase])
        epoch_acc_vgg16 = 100 * running_corrects_vgg16 / len(image_datasets[phase])
        epoch_loss_resnet50 = running_loss_resnet50 * 16/ len(image_datasets[phase])
        epoch_acc_resnet50 = 100 * running_corrects_resnet50 / len(image_datasets[phase])
        epoch_blending_acc = 100 * blending_running_corrects / len(image_datasets[phase])
        
        print('Epoch BLENDING ACC :{:.4f}'.format(epoch_blending_acc))
        

# RNN(Recurrent Neural Network)

In [ ]:
import torchvision
import torch
from torchvision import datasets,transforms
from torch.autograd import Variable
import matplotlib.pyplot as plt
%matplotlib inline

transform = transforms.Compose(  [transforms.ToTensor(),transforms.Normalize(mean=[0.5],std=[0.5])] )
dataset_train = datasets.MNIST(root='./data',transform = transform,train=True,download=True)
dataset_test  = datasets.MNIST(root='./data',transform = transform,train=False)

train_load = torch.utils.data.DataLoader(dataset=dataset_train,batch_size=64,shuffle=True)
test_load  = torch.utils.data.DataLoader(dataset=dataset_test, batch_size=64,shuffle=True)

images,label = next(iter(train_load))

images_example = torchvision.utils.make_grid(images)
images_example = images_example.numpy().transpose(1,2,0) #因为在plt.imshow在现实的时候输入的是(imagesize,imagesize,channels）imshow中，参数img的格式为（channels,imagesize,imagesize）,

mean = [0.5,0.5,0.5]
std = [0.5,0.5,0.5]
images_example = images_example*std + mean

plt.imshow(images_example)
plt.show()

In [ ]:
# 循环神经网络主要用于处理序列(Sequences)问题，当然RNN也可以解决分类问题
#CNN的input维度，output维度，模型结构层次都是固定的。
#由于在RNN中可以随意控制输入数据及输出数据的数量，具有非常大的灵活性。
import torch

class RNN(torch.nn.Module):
    def __init__(self):
        super(RNN,self).__init__()
        self.rnn = torch.nn.RNN(input_size=28,hidden_size=128,num_layers=1,batch_first=True)
        self.output = torch.nn.Linear(128,10)
    
    def forward(self,input):
        output,_ = self.rnn(input,None)
        output = self.output(output[:,-1,:])
        return output
#RNN中输入层到输出层默认的维度是(seq,batch,feature)即(序列长度，批次数量，输入或输出特征数)

model = RNN()
print('===================================================')
print(model)
print('===================================================')

model = model.cuda()

optimizer = torch.optim.Adam(model.parameters())
loss_f = torch.nn.CrossEntropyLoss()
loss_f = loss_f.cuda()

epoch_n = 10

for epoch in range(epoch_n):
    running_loss = 0.0
    running_correct = 0
    testing_correct = 0
    print('---------------------------------------------------')
    print('Epoch:{}/{}'.format(epoch,epoch_n))
    
    for data in train_load:
        x_train,y_train = data
        x_train = x_train.view(-1,28,28)
        x_train,y_train = x_train.cuda(),y_train.cuda()
        
        y_pred = model(x_train)
        loss = loss_f(y_pred,y_train)
        _,pred = torch.max(y_pred.data,1)
        
        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        
        running_loss += loss.item()
        running_correct += torch.sum(pred==y_train.data)
    for data in test_load:
        x_test,y_test = data
        x_test = x_test.view(-1,28,28)
        x_test,y_test = x_test.cuda(),y_test.cuda()
        outputs = model(x_test)
        _,pred = torch.max(outputs.data,1)
        testing_correct += torch.sum(pred == y_test.data)
    print('Loss:{:.4f} Acc:{:.4f}%---test Acc:{:.4f}'.format(running_loss/len(dataset_train),100*running_correct/len(dataset_train),100*testing_correct/len(dataset_test)))
